In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


In [6]:
batch_size=100
num_unrollings=100

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // (2*batch_size)
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    
    # Generate a batch taking only one char from each segment. Next batch will take the
    # following char in each segment and so on.
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      # use the char id to set the corresponding position in the row to 1
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0 
      # increment segment cursor +1 so the next batch picks the next char of each segment
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  
  # To reconstruct a word, we need to pick the char in the same position of each batch
  # and concatenate them.
  # On the first iteration, the comprehension simply transforms ids into chars and 
  # copies them into s. On every subsequent iteration the comprehension will concatenate
  # what's already in s with what's in the next batch in the same position.
  # In this way it reconstruct the words by joining together the chars that were in 
  # different batches
  # We have n batches with m chars on each. This will generate m strings with n chars on each
  s = [''] * batches[0].shape[0]
  for b in batches:
    
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchists advocate social relations based upon voluntary association of autonomous individuals m', 'r have discovered roger bacon has also been attributed with originating the search for the philosophe', 'ormer nike surface to air missile site that same year oil was discovered in prudhoe bay on the arctic', 'r high regard for personal honor for their clan loyalty and for their readiness to carry and use arms', 'this use of the term is much more common in europe characteristics dragon ball z is one of the most p', 'n centimetres of mifune s body other stories include demanding a stream be made to run in the opposit', ' soviets from berlin speer even confessed to hitler shortly before the dictator s suicide that he had', 'e culture of organs and worked with lindbergh in the mid one nine three zero s to create the perfusio', 'a combination of glaukos which can be translated as gleaming silvery and later as bluish green or gra', ' cit ii six six two seven two four contests both thes

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [14]:
num_nodes = 128
graph = tf.Graph()
with graph.as_default():
    
  
  # Parameters:
  x = dict()
  m = dict()
  bl = dict()
  # Layer 1
  x["layer 1"] = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  m["layer 1"] = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  bl["layer 1"] = tf.Variable(tf.zeros([1, 4*num_nodes]))
  # Layer 2
  x["layer 2"] = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  m["layer 2"] = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  bl["layer 2"] = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
  # Variables saving state across unrollings for each layer.
  saved_output = dict()
  saved_state = dict()
  # Layer 1
  saved_output["layer 1"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state["layer 1"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Layer 2
  saved_output["layer 2"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state["layer 2"] = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  
  
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state, layer):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    lstm_tensor = tf.matmul(i, x[layer]) + tf.matmul(o, m[layer]) + bl[layer]
    
    input_gate = tf.sigmoid(lstm_tensor[:, :num_nodes])
    forget_gate = tf.sigmoid(lstm_tensor[:, num_nodes:2*num_nodes])
    update = lstm_tensor[:, 2*num_nodes:3*num_nodes]
    output_gate = tf.sigmoid(lstm_tensor[:, 3*num_nodes:])
    
    state = forget_gate * state + input_gate * tf.tanh(update)
    return output_gate * tf.tanh(state), state

  
  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.
    
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output["layer 1"]
  state = saved_state["layer 1"]
  output_layer_2 = saved_output["layer 2"]
  state_layer_2 = saved_state["layer 2"]
    
  for i in train_inputs:
    output, state = lstm_cell(i, output, state, "layer 1")
    #outputs.append(tf.nn.dropout(output, keep_prob=0.8))
    output_layer_2, state_layer_2 = lstm_cell(tf.nn.dropout(output, keep_prob=0.9), 
                                              output_layer_2, state_layer_2, "layer 2")
    outputs.append(tf.nn.dropout(output_layer_2, keep_prob=0.9))

  # State saving across unrollings.
  with tf.control_dependencies([saved_output["layer 1"].assign(output),
                                saved_state["layer 1"].assign(state),
                                saved_output["layer 2"].assign(output_layer_2),
                                saved_state["layer 2"].assign(state_layer_2)]):
    
    
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = dict()  
  saved_sample_state = dict()
  saved_sample_output["layer 1"] = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state["layer 1"] = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_output["layer 2"] = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state["layer 2"] = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output["layer 1"].assign(tf.zeros([1, num_nodes])),
    saved_sample_state["layer 1"].assign(tf.zeros([1, num_nodes])),
    saved_sample_output["layer 2"].assign(tf.zeros([1, num_nodes])),
    saved_sample_state["layer 2"].assign(tf.zeros([1, num_nodes])))

  sample_output = dict()
  sample_state = dict()
  sample_output["layer 1"], sample_state["layer 1"] = lstm_cell(
    sample_input, saved_sample_output["layer 1"], saved_sample_state["layer 1"], "layer 1")
    
  sample_output["layer 2"], sample_state["layer 2"] = lstm_cell(
    sample_output["layer 1"], saved_sample_output["layer 2"], saved_sample_state["layer 2"], "layer 2")

  with tf.control_dependencies([saved_sample_output["layer 1"].assign(sample_output["layer 1"]),
                                saved_sample_state["layer 1"].assign(sample_state["layer 1"]),
                                saved_sample_output["layer 2"].assign(sample_output["layer 2"]),
                                saved_sample_state["layer 2"].assign(sample_state["layer 2"])]):
        
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output["layer 2"], w, b))




In [15]:
num_steps = 14001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.304554 learning rate: 10.000000
Minibatch perplexity: 27.24
wdhceaemgsviweqsntkmifnzch poyhcclrhktactlpfeoyoujczartnpyjamzzymqgionzdyjtrprhf
jeczt b wzfgaklkvrdqvkekl sgingneddgiqe  ryavbljvlpqobartdyjxzbkonqnpqqoofsypznk
zxpwjqus mtoffhyfnlznvlwwjnwvlkcfjvhurfopskjm titlliqgnoeewwfksj uczoxqrayxwxyoj
pqlkzwlxnjpyifrjjwnp scfwdmklbyryxzvtcspchyxvseyglllixyqjfodbmydtnjmehimeiftpb j
surauhqnfdtvixwdrbqqwzrvulzryxcmelkpstwebqujwnkbdbrzhx seiodwdgovfepryndrkdzldxj
Validation set perplexity: 27.39
Average loss at step 100: 2.922783 learning rate: 10.000000
Minibatch perplexity: 16.41
Validation set perplexity: 16.03
Average loss at step 200: 2.711830 learning rate: 10.000000
Minibatch perplexity: 13.27
Validation set perplexity: 13.96
Average loss at step 300: 2.489787 learning rate: 10.000000
Minibatch perplexity: 10.86
Validation set perplexity: 10.56
Average loss at step 400: 2.344618 learning rate: 10.000000
Minibatch perplexity: 10.27
Validation se